# Import the libraries for face recognition

In [ ]:
import smtplib as s
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import joblib
import os
from email.message import EmailMessage
import pywhatkit
import datetime
import subprocess

# Loading haar face classifier 👇

In [2]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
n = int(input("Enter no. of user: "))

# Creating n number of users 👇

In [4]:
USER = [i+1 for i in range(0,n)]

In [ ]:
USER

# Taking input of user names 👇

In [ ]:
USER_NAME = []     

for i in range(0,n):
    name = input("Enter user " + str(i+1) + " name : ")
    USER_NAME.append(name)

# Creating function that detects faces and return the cropped face. If no face detected, it will return the input img. 👇

In [7]:

def face_extractor(img):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3,5)
    
    if faces is (): 
        return None
    
    #crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        
    return cropped_face

<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-7-44805affd518>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


# Creating a function which will create folder for both users 👇

In [8]:
def change_dir (i):
    os.chdir("D:\task6\faces")
    os.system("mkdir "+ str(USER_NAME[i]))

# Creating function which will initialize webcam and collect 100 samples of img from webcam 👇

In [9]:
def capture(i):
    cap = cv2.VideoCapture(0)
    count = 0

    while True:
    
        ret, photo = cap.read()
        if face_extractor(photo) is not None:
            count += 1
            face = cv2.resize(face_extractor(photo),(200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  #training would be faster for B/W img
        
        #save file in specified directory with unique name
            change_dir(i)
            file_name_path = 'D:\task6\faces'+ str(USER_NAME[i]) +"/"+ str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)
        
        #put count on img and display live count
            cv2.putText(face, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255),2)
            cv2.imshow('Face cropper', face)
        
        
        else:
            pass
    
        if cv2.waitKey(1) == 13 or count == 100:  #13 is for ENTER key
            break
        

    cap.release()
    cv2.destroyAllWindows()
    print("Sample Collection for "+ str(USER_NAME[i]) +" completed")

# Sample collection from both user 👇

In [ ]:
for i in range(0,n):
    capture(i)

# This function will help to get the training data and then it will train both models 👇

In [10]:
LBPH_obj = [None, None]


def train_model(j):
    data_path = 'D:\task6\faces'+ str(USER_NAME[j]) + "/"     #. is used for current dir
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]


    #create arrays for training data and labels
    Train_data , Labels = [], []

    #open training images in our datapath
    #create numpy array for training data

    for i , files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Train_data.append(np.asarray(images, dtype = np.uint8))
        Labels.append(i)


    #create a numpy array for both training data and lebels
    Labels = np.asarray(Labels, dtype = np.int32)

    
    #initialize facial recognizer
    #model = cv2.face.createLBPHFaceRecognizer()
    # Note : for opencv 3.0 use cv2.face.createLBPHFaceRecognizer()
    #pip install opencv-contrib-python
    #model = cv2.createLBPHFaceRecognizer()

    LBPH_obj[j] = cv2.face_LBPHFaceRecognizer.create()

    #train the model
    LBPH_obj[j].train(np.asarray(Train_data), np.asarray(Labels))
    print("Model for " +str(USER_NAME[j])+ " trained successfully")

# Calling function to train the models 👇

In [ ]:
for j in range(0,n):
    train_model(j)

# This function will help to get the region of interest. 👇

In [12]:
def face_detector(img, size = 0.5):
    
    #convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y),(x+w, y+h),(0,255,0),2)
        region_of_interest = img[y:y+h, x:x+w]
        region_of_interest = cv2.resize(region_of_interest, (200,200))
        
    return img, region_of_interest

<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-12-83000bab0f05>:6: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is():


# Function for sending email 👇

In [ ]:
def email():
    send_email = "sampreethi66@gmail.com"

    with open("D:\task6\faces") as file:                       #reading password
        pswrd = file.read()

    msg = EmailMessage()

    msg["Subject"] = 'Face recognition'
    msg['From'] = send_email
    msg['To'] = 'sampreethi66@gmail.com'

    msg.set_content("This is face of sampreethi")

    server = s.SMTP_SSL("smtp.gmail.com", 465)
    server.login(send_email,pswrd)
    print("Login Success!!")


    server.send_message(msg)
    print("Email has been sent successfully")

    server.close()

# Function for sending whatsapp msg 👇

In [ ]:
def whatsapp():
    
    file = open("D:\task6\faces")                     
    ph_no = file.read()
    pywhatkit.sendwhatmsg_instantly(ph_no, "This msg was sent using python code")

# This function will create a new EC2 instance and EBS volume and then attach both 👇

In [ ]:
def instance_vol():
    i_id = subprocess.getoutput("aws ec2 run-instances --image-id ami-0320b895e6e618ca3 --instance-type t2.micro --count 1 --subnet-id subnet-8b93c9c7  --key-name awskey --query Instances[*].[InstanceId] --output text")
    print("EC2 Instance with instance id : {} created successfully".format(i_id))
    v_id = subprocess.getoutput("aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone ap-south-1a --query VolumeId --output text")
    print("Created 5GB EBS Volume with volume id : {} created".format(v_id))
    attach()

In [ ]:
def attach():
    vid = input("Enter volume id: ")
    iid = input("Enter instance id: ")
    subprocess.getoutput("aws ec2 attach-volume --volume-id " + vid +  " --instance-id " + iid + " --dev /dev/sdf")
    print("EBS volume Attached successfully to the newly created instance")

# Face recognition part 👇

In [ ]:

cap = cv2.VideoCapture(0)
def capture():
    count_1 = 0
    count_2 = 0
    
    while True:
    
        ret, photo = cap.read()
        image, face = face_detector(photo)
    
        try:
            k = 0
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
            #pass face to prediction model
            #"result" will be a tuple contaning the label and the confidence value
            results = LBPH_obj[k].predict(face)
            
         
            if results[1] < 500:                        #results[1] ==> confidence value
                confidence = int(100*(1 - (results[1])/400))
                
            if confidence > 88:
                count_1 += 1
                display_string = str(confidence) + ' % Confident she is sampreethi'
                cv2.putText(image, display_string, (100,120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
                cv2.putText(image, "Email & whatsapp msg done", (100,400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow("Face Recognition", image)
                if count_1 == 10:
                    email()
                    whatsapp()
                    
                  

            k += 1                                 #switching to 2nd model
            display_string= ""
            results = LBPH_obj[k].predict(face)
                
            if results[1] < 500:
                confidence = int(100*(1 - (results[1])/400))            
        
            if confidence > 88:
                count_2 += 1
                txt ="Instance launched"
                display_string = str(confidence) + ' % Confident he is Mukul'
                cv2.putText(image, display_string, (80,100), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
                cv2.putText(image, txt, (100,400), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                
                if count_2 == 10:
                    instance_vol()
                    
                cv2.imshow("Face Recognition", image)
                
        except:
            cv2.imshow("Face Recognition", image)
            pass
        
        if cv2.waitKey(1) == 13:
            break
            
            
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
capture()